## Steam 数据加载与预处理

核心逻辑：

1. 处理 tags_list: 它是字符串格式的列表 "[1663, 1774]"，需要还原并补齐长度。

2. 处理 price: 归一化。

3. 处理 item_id: 建立索引映射。

In [ ]:
# ==========================================
# 🎮 Steam 数据加载器 (Data Loader)
# ==========================================
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from deepctr_torch.inputs import SparseFeat, DenseFeat, VarLenSparseFeat

def load_steam_data(csv_path):
    print(f"Loading Steam data from: {csv_path} ...")
    data = pd.read_csv(csv_path)
    
    # 1. 【关键】将字符串格式的 list 还原为真正的 list
    # CSV里存的是 "[1663, 1774]" (String)，模型要的是 [1663, 1774] (List)
    data['tags_list'] = data['tags_list'].apply(lambda x: ast.literal_eval(x))

    # 2. 特征编码 (Encoding)
    # Item ID: 离散特征 -> 0, 1, 2...
    lbe = LabelEncoder()
    data['item_id_idx'] = lbe.fit_transform(data['item_id'])
    
    # Price: 连续特征 -> 0.0 ~ 1.0
    mms = MinMaxScaler(feature_range=(0, 1))
    data['price_norm'] = mms.fit_transform(data[['price']])

    # 3. 序列特征处理 (Padding)
    # DeepFM 要求 tags 输入必须定长。假设最大长度为 5。
    # [1663, 1774] -> [1663, 1774, 0, 0, 0]
    MAX_TAG_LEN = 5
    MAX_TAG_ID = 10000 # 假设 Tag ID 不会超过这个数，实际可以用 max(all_tags)+1
    
    tags_list = list(data['tags_list'])
    # padding='post' 表示在后面补0
    tags_padded = pad_sequences(tags_list, maxlen=MAX_TAG_LEN, padding='post', truncating='post', value=0)

    # 4. 定义 DeepFM 特征配置 (Feature Columns)
    # 告诉模型：哪些是离散的，哪些是连续的，哪些是变长的
    fixlen_feature_columns = [
        SparseFeat(name='item_id_idx', vocabulary_size=data['item_id_idx'].max() + 1, embedding_dim=16),
        DenseFeat(name='price_norm', dimension=1)
    ]
    
    varlen_feature_columns = [
        VarLenSparseFeat(
            SparseFeat('tags', vocabulary_size=MAX_TAG_ID, embedding_dim=16),
            maxlen=MAX_TAG_LEN, combiner='mean', length_name=None
        )
    ]
    
    linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

    # 5. 组装输入字典 (Input Dict)
    model_input = {
        'item_id_idx': data['item_id_idx'].values,
        'price_norm': data['price_norm'].values,
        'tags': tags_padded
    }
    
    # Label
    target = data['label'].values
    
    return model_input, linear_feature_columns, dnn_feature_columns, target

# --- 测试运行 ---
# 假设你保存的文件名是 steam_data.csv
# steam_input, steam_linear, steam_dnn, steam_y = load_steam_data('steam_data.csv')
# print("Steam Input Shapes:", {k: v.shape for k, v in steam_input.items()})

## Steam 模型训练 (Training)

In [ ]:
import torch
from deepctr_torch.models import DeepFM
from deepctr_torch.callbacks import EarlyStopping

# 1. 准备数据
# 确保你已经运行了 Cell 2 定义了 load_steam_data
steam_csv_path = '../data/steam/train_data_Hardcore_Gamer.csv' # 你的文件名
input_dict, linear_cols, dnn_cols, target = load_steam_data(steam_csv_path)

# 2. 检查设备 (有显卡用显卡，没显卡用 CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 使用设备: {device}")

# 3. 初始化 DeepFM 模型
# 关键点：dnn_hidden_units 定义了神经网络的层数和宽度
model_steam = DeepFM(linear_feature_columns=linear_cols, 
                     dnn_feature_columns=dnn_cols, 
                     task='binary', 
                     dnn_hidden_units=(128, 128), 
                     device=device)

# 4. 编译模型
# 优化器用 Adam，损失函数用 二元交叉熵 (binary_crossentropy)
model_steam.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy', 'auc'])

# 5. 开始训练 (Fit)
history_steam = model_steam.fit(input_dict, target, 
                                batch_size=16, # 数据少，batch_size 设小点
                                epochs=10, 
                                verbose=2, 
                                validation_split=0.2) # 划 20% 做验证集

print("🎉 Steam 模型训练完成！")

## 模型保存 (Model Saving)

In [ ]:
# ==========================================
# 💾 模型权重保存
# ==========================================
import torch
import os

# 1. 定义保存路径 (建议放在当前工作目录)
steam_model_path = 'deepfm_steam_weights.pth'

# 2. 保存 Steam 模型
# 只有当模型已经训练过 (model_steam 存在) 才保存
if 'model_steam' in locals():
    torch.save(model_steam.state_dict(), steam_model_path)
    print(f"✅ Steam 模型权重已保存至: {steam_model_path}")
else:
    print("⚠️ 未检测到 model_steam，跳过保存。")

print("\n提示: 在 ModelScope 左侧文件列表中应该能看到这两个 .pth 文件了。")

## 加载权重并启动 API 服务 (Loading & Serving)

In [ ]:
# ==========================================
# 🚀 模型加载与 API 服务启动
# ==========================================
import torch
import pandas as pd
import numpy as np
from deepctr_torch.models import DeepFM
from flask import Flask, request, jsonify
import threading

# 1. 重新定义一些基础配置 (防止重启内核后变量丢失)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
STEAM_CSV = '../data/steam/train_data_Hardcore_Gamer.csv'   # 确保文件名对

# 2. 定义加载函数 (复用之前的逻辑，只为了获取特征列配置来初始化模型)
#    注意：这里我们需要 import 之前定义的 load_steam_data
#    如果你重启了内核，请确保把"数据预处理"那个 Cell 先运行一遍，让函数生效。

print("🔄 正在初始化服务，重新构建模型结构...")

# --- A. 加载 Steam 模型 ---
# 1. 获取特征结构 (不需要真实 Input 数据，只需要 Feature Columns)
steam_input, steam_linear, steam_dnn, _ = load_steam_data(STEAM_CSV)
# 2. 初始化空模型
service_model_steam = DeepFM(steam_linear, steam_dnn, task='binary', dnn_hidden_units=(128, 128), device=DEVICE)
# 3. 加载权重
try:
    service_model_steam.load_state_dict(torch.load('deepfm_steam_weights.pth', map_location=DEVICE))
    service_model_steam.eval() # 切换到评估模式
    print("✅ Steam 模型加载成功！")
except FileNotFoundError:
    print("❌ 未找到 Steam 权重文件，服务将无法处理 Steam 请求。")

# ==========================================
# 3. 搭建 Flask API
# ==========================================
app = Flask(__name__)

# 定义推荐逻辑 (封装好的工具函数)
def get_recommendation(model, input_data, csv_path, top_k=3):
    # 1. 预测
    pred_ans = model.predict(input_data, batch_size=256)
    # 2. 读原始文件用于展示
    df = pd.read_csv(csv_path)
    df['score'] = pred_ans
    # 3. 排序
    top_items = df.sort_values(by='score', ascending=False).head(top_k)
    # 4. 格式化输出
    results = []
    for _, row in top_items.iterrows():
        results.append({
            "id": str(row['item_id']),
            "title": row['title'],
            "score": float(row['score']),
            "cover": row.get('cover_url', row.get('pdf_url', '')),
            "tags": row.get('tag_names', row.get('category', ''))
        })
    return results

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "device": DEVICE})

@app.route('/recommend', methods=['POST'])
def recommend():
    """
    API 接口
    Input JSON: { "type": "steam", "top_k": 3 }
    """
    data = request.json
    rec_type = data.get('type', 'steam')
    top_k = data.get('top_k', 3)
    
    try:
        if rec_type == 'steam':
            # 注意：实际生产中，input_data 应该根据 user_id 动态生成
            # 这里演示用，直接使用全量 input_data 进行重排
            res = get_recommendation(service_model_steam, steam_input, STEAM_CSV, top_k)
        else:
            return jsonify({"error": "Unknown type"}), 400
            
        return jsonify({
            "code": 200,
            "type": rec_type,
            "data": res
        })
    except Exception as e:
        return jsonify({"code": 500, "error": str(e)})

# ==========================================
# 4. 启动服务 (在 Notebook 中后台运行)
# ==========================================
def run_app():
    # 端口设置为 6006 或 5000
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# 使用线程启动，避免阻塞 Notebook 的主进程，这样你还能继续跑下面的测试代码
t = threading.Thread(target=run_app)
t.daemon = True
t.start()

print("🚀 服务已启动！监听端口 5000...")
print("你可以运行下一个 Cell 来测试接口。")

## 测试服务 (Client Test)

In [ ]:
# ==========================================
# 🧪 接口测试 (模拟 Dify 调用)
# ==========================================
import requests
import time

# 等待一秒让服务完全启动
time.sleep(2)

url = "http://127.0.0.1:5000/recommend"

# 1. 测试 Steam 推荐
print("📡 请求 Steam 推荐...")
payload_steam = {"type": "steam", "top_k": 2}
try:
    resp = requests.post(url, json=payload_steam)
    print("Steam Response:", resp.json())
except Exception as e:
    print("Steam Request Failed:", e)

print("-" * 30)

## 最终服务代码 Cell (直接运行)

In [ ]:
# ==========================================
# 🎮 Steam DeepFM Server (Port 5000)
# ==========================================
import torch
import pandas as pd
import numpy as np
from deepctr_torch.models import DeepFM
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading

# ⚠️⚠️⚠️ 填入你的 Ngrok Token ⚠️⚠️⚠️
NGROK_TOKEN = "这里粘贴你的_Ngrok_Token" 
PORT = 5000 # Steam 服务运行在 5000 端口

# ================= 1. 加载 Steam 模型 =================
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
STEAM_CSV = '../data/steam/train_data_Hardcore_Gamer.csv'

print("🔄 正在初始化 Steam 模型...")

# 假设 load_steam_data 已经在前面的 Cell 定义好了
try:
    steam_input, steam_linear, steam_dnn, _ = load_steam_data(STEAM_CSV)
    
    # 初始化模型结构
    model_steam = DeepFM(steam_linear, steam_dnn, task='binary', dnn_hidden_units=(128, 128), device=DEVICE)
    
    # 加载权重
    model_steam.load_state_dict(torch.load('deepfm_steam_weights.pth', map_location=DEVICE))
    model_steam.eval()
    print("✅ Steam 模型加载成功！Ready to serve.")
except Exception as e:
    print(f"❌ Steam 模型加载失败: {e}")
    # 这里为了不报错退出，你可以选择抛出异常或继续

# ================= 2. 定义 Flask 应用 =================
app_steam = Flask(__name__)

def get_steam_recs(top_k=3):
    # 1. 预测
    pred_scores = model_steam.predict(steam_input, batch_size=256)
    
    # 2. 读取元数据
    df = pd.read_csv(STEAM_CSV)
    df['score'] = pred_scores
    
    # 3. 排序
    top_items = df.sort_values(by='score', ascending=False).head(top_k)
    
    # 4. 格式化
    results = []
    for _, row in top_items.iterrows():
        results.append({
            "title": row['title'],
            "score": float(row['score']),
            "type": str(row.get('tag_names', 'Unknown')),
            "cover": str(row.get('cover_url', ''))
        })
    return results

@app_steam.route('/recommend', methods=['POST'])
def recommend_steam():
    try:
        req = request.json
        top_k = req.get('top_k', 3)
        print(f"🎮 [Steam] 收到请求, Top {top_k}")
        
        data = get_steam_recs(top_k)
        return jsonify({"status": "success", "service": "steam", "recommendations": data})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

# ================= 3. 启动服务 =================
def start_steam_server():
    ngrok.set_auth_token(NGROK_TOKEN)
    # 杀掉旧进程防止冲突
    ngrok.kill()
    
    public_url = ngrok.connect(PORT, bind_tls=True).public_url
    print("\n" + "="*50)
    print(f"🎮 Steam 服务已启动！API 地址: {public_url}/recommend")
    print("="*50 + "\n")
    
    app_steam.run(port=PORT, use_reloader=False)

if __name__ == '__main__':
    t = threading.Thread(target=start_steam_server)
    t.daemon = True
    t.start()